# 1. 选择文件夹

In [2]:
import os

In [3]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if '替代' in file:
        alternate = './' + file
    elif '断货改价' in file:
        base = './' + file
    elif '销量' in file:
        sales = './' + file

print(f'\talternate: \t{alternate}')
print(f'\tbase: \t\t{base}')
print(f'\tsales: \t\t{sales}')

	alternate: 	./\整理替代料表.ipynb


NameError: name 'base' is not defined

# 2. 替代料表

In [4]:
import pandas as pd

In [5]:
df_alternate = pd.read_excel(alternate,
                             header=0).fillna('')

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,
1,USGS43640B,USGS43640,,,,,
2,USBDA04106,USBDA04106X,,,,,
3,USGS6310TNZB,USGS6310TZ,USGS6310T,USGS6310,USGS6310NZ,USGS6310TNZ,
4,USGS61560B,USGS61560,,,,,
...,...,...,...,...,...,...,...
19247,USDS76733-C,USDS76733,,,,,
19248,USDS76734-C,USDS76734,,,,,
19249,USDS76803-C,USDS76803,,,,,
19250,USDS76934-C,USDS76934,,,,,


In [8]:
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,,0
1,USGS43640B,USGS43640,,,,,,1
2,USBDA04106X,USBDA04106,,,,,,2
3,USGS6310TZ,USGS6310TNZB,USGS6310TNZ,USGS6310T,USGS6310NZ,USGS6310,,3
4,USGS61560B,USGS61560,,,,,,4
...,...,...,...,...,...,...,...,...
19238,USDS76733-C,USDS76733,,,,,,19238
19239,USDS76734-C,USDS76734,,,,,,19239
19240,USDS76803-C,USDS76803,,,,,,19240
19241,USDS76934-C,USDS76934,,,,,,19241


In [9]:
list_df = []
for i in ['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']:
    df_temp =  df_alternate[[i,
                             'row']]
    df_temp.columns = ['SKU',
                       'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_position = pd.concat(list_df).reset_index(drop=True)

df_position

,SKU,row
0,USCA171876D-3,0
1,USGS43640B,1
2,USBDA04106X,2
3,USGS6310TZ,3
4,USGS61560B,4
...,...,...
41055,USBM00215,2886
41056,USIC02680-6,11675
41057,USIC02680,11678
41058,USFP3615M0,17373


# 3. 销量计算

In [10]:
df = pd.read_excel(base,
                   header=1)[['SKU']]

df

,SKU
0,USFP2332S0Z-C
1,USFP2344M0Z-C
2,USFP2359M0Z-C
3,USFP2442M0Z-C
4,USFP2455M0Z-C
...,...
10786,USFP2307M0-C
10787,USGS33000-1
10788,USOC19025-C
10789,USOC19142-C


In [11]:
df = pd.merge(left=df, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna('')
df = df[['SKU',
         'row']]

df

,SKU,row
0,USFP2332S0Z-C,17295
1,USFP2344M0Z-C,17296
2,USFP2359M0Z-C,17304
3,USFP2442M0Z-C,16772
4,USFP2455M0Z-C,16987
...,...,...
10786,USFP2307M0-C,
10787,USGS33000-1,
10788,USOC19025-C,
10789,USOC19142-C,


In [12]:
df_sales = pd.read_excel(sales,
                         header=0,
                         usecols='D, F')

df_sales = df_sales.groupby(df_sales['SKU'], as_index=False).sum()

df_sales

,SKU,购买数量
0,USABS10340,1
1,USABS11020,1
2,USABS11430,1
3,USABS13190,1
4,USABS15760,1
...,...,...
1713,USWS50157-C,1
1714,USWS59271,1
1715,USWS5AA02,1
1716,USWS8A068,1


In [13]:
df = pd.merge(left=df, right=df_sales, how='left', left_on='SKU', right_on='SKU').fillna(0)

df

,SKU,row,购买数量
0,USFP2332S0Z-C,17295,0.0
1,USFP2344M0Z-C,17296,0.0
2,USFP2359M0Z-C,17304,0.0
3,USFP2442M0Z-C,16772,0.0
4,USFP2455M0Z-C,16987,0.0
...,...,...,...
10786,USFP2307M0-C,,0.0
10787,USGS33000-1,,0.0
10788,USOC19025-C,,0.0
10789,USOC19142-C,,0.0


In [14]:
df_sales = pd.merge(left=df_sales, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna('')
df_sales = df_sales[df_sales['row'] != ''][['row', '购买数量']].copy()
df_sales = df_sales.groupby([df_sales['row']], as_index=False).sum()

df_sales

,row,购买数量
0,1,4
1,1000,1
2,10054,1
3,10101,1
4,10103,1
...,...,...
719,9845,1
720,9859,1
721,9918,1
722,9946,2


In [15]:
df = pd.merge(left=df, right=df_sales, how='left', left_on='row', right_on='row', suffixes=('_sales_1', '_sales_2')).fillna('')

df

,SKU,row,购买数量_sales_1,购买数量_sales_2
0,USFP2332S0Z-C,17295,0.0,
1,USFP2344M0Z-C,17296,0.0,
2,USFP2359M0Z-C,17304,0.0,
3,USFP2442M0Z-C,16772,0.0,
4,USFP2455M0Z-C,16987,0.0,
...,...,...,...,...
10786,USFP2307M0-C,,0.0,
10787,USGS33000-1,,0.0,
10788,USOC19025-C,,0.0,
10789,USOC19142-C,,0.0,


In [16]:
df['购买数量'] = [df.loc[i, '购买数量_sales_2'] if df.loc[i, '购买数量_sales_2'] != '' else df.loc[i, '购买数量_sales_1'] for i in range(len(df))]

df = df[['SKU', '购买数量']]

df.to_excel('./date 销量.xlsx', index=False)